# Agent-Based Simulation: Directed Search 

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import random 
import pandas as pd
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.optimize import least_squares
from scipy.stats import beta
from scipy.stats import bernoulli
from scipy.stats import cumfreq, beta
from utils import simulate_game

%matplotlib inline
plt.style.use('../notebook.mplstyle')

SEED = 1 

## Exogenous Parameters


In [2]:
# Setting exogenous parameters
def reset_exog_params():
    global Bm, Bw, bm_vals, bw_vals, δ, um, uw, Fm, Fw, λm, λw
    Bm = 10
    Bw = 10
    bm_vals = range(1,Bm+1) 
    bw_vals = range(1,Bw+1)
    
    δ = 0.97
    um = lambda θ : θ 
    uw = lambda θ : θ
    Fm = beta(3,3)
    Fw = beta(3,3)
    λm = 50
    λw = 50

def exog_params():
    return (Bm, Bw, bm_vals, bw_vals, δ, um, uw, Fm, Fw, λm, λw)

## Steady State with Cutoff Strategies

In [3]:
def steady_state_men(μ): 
    # Computing z's
    zm = []
    for b in bm_vals:
        if b==Bm:
            zm.append(1-δ*Fw.cdf(μ[Bm-1]))
        else:    
            z=1
            for i in range(1, Bm-b+1): 
                z *= ((δ*(1-Fw.cdf(μ[Bm-i])))/(1-δ*Fw.cdf(μ[Bm-i-1])))   
            zm.append(z)  
    # Computing steady-state mass 
    Nm = (λm) * ((zm[Bm-1] - δ * zm[0] * (1 - Fw.cdf(μ[0]))) / ((1-δ) * zm[Bm-1]))

    # Computing steady state distribution over budgets
    Pbm = [((λm) / (Nm * zm[Bm-1])) * zm[b] for b in range(Bm-1)]
    Pbm.append(((λm) / (Nm * zm[Bm-1])))  

    return Nm, Pbm 

In [4]:
def steady_state_women(ω):
    # Computing z's
    zw = []
    for b in bw_vals:
        if b==Bw:
            zw.append(1-δ*Fm.cdf(ω[Bw-1]))
        else:    
            z=1
            for i in range(1, Bw-b+1): 
                z *= ((δ * (1-Fm.cdf(ω[Bw-i])))/(1-δ*Fm.cdf(ω[Bw-i-1])))
            zw.append(z)    
    
    # Computing steady-state mass
    Nw = (λw) * ((zw[Bw-1] - δ * zw[0] * (1 - Fm.cdf(ω[0]))) / ((1-δ) * zw[Bw-1]))

    # Computing steady state distribution over budgets  
    Pbw = [((λw) / (Nw * zw[Bw-1])) * zw[b] for b in range(Bw-1)] 
    Pbw.append(((λw) / (Nw * zw[Bw-1])))  

    return Nw, Pbw

In [5]:
def steady_state(μ, ω, verbose=False):
    # Computing masses and distributions
    Nm, Pbm = steady_state_men(μ)
    Nw, Pbw = steady_state_women(ω)
    
    # Computing tightness and alpha
    if Nw>Nm:
        τm = 1
    else:
        τm = Nw/Nm 
        
    τw = τm *(Nm/Nw) 
    αm = (τm*δ)/(1-δ*(1-τm))
    αw = (τw*δ)/(1-δ*(1-τw))
    return Nm, Nw, Pbm, Pbw, τm, τw, αm, αw

## Two-Sided Search Equilibrium Conditions 


In [6]:
# Optimality conditions
def SSE(x): 
    # Compute steady state 
    Nm, Nw, Pbm, Pbw, τm, τw, αm, αw = steady_state(x[:Bm], x[Bm:])
    
    Um = um
    Uw = uw
    # Computing ex-ante expected utilities 
    
    # Initialysing system of equilibrium equations
    E = np.empty(2*Bm + 2*Bw + 2)
    
    # Initial conditions 
    E[0] = (Um(x[0]) 
            - αm * Um(x[0]) * Fw.cdf(x[0]) 
            - αm * intg.quad(lambda t: Um(t) * Fw.pdf(t), x[0], 1)[0]) 
    
    E[Bm] = (Uw(x[Bm]) 
            - αw * Uw(x[Bm]) * Fm.cdf(x[Bm]) 
            - αw * intg.quad(lambda t: Uw(t) * Fm.pdf(t), x[Bm], 1)[0]) 
    
    # Intertemporal optimality conditions for men
    for b in range(1,Bm):
        E[b] = (Um(x[b]) 
                - αm * Um(x[b]) * Fw.cdf(x[b]) 
                - αm * Um(x[b-1])*(1-Fw.cdf(x[b-1])) 
                - αm * intg.quad(lambda t : Um(t) * Fw.pdf(t), x[b], x[b-1])[0])
    
    # Intertemporal optimality conditions for women
    for b in range(1,Bw):
        E[Bm+b] = (Uw(x[Bm+b]) 
                - αw * Uw(x[Bm+b]) * Fm.cdf(x[Bm+b]) 
                - αw * Uw(x[Bm+b-1])*(1-Fm.cdf(x[Bm+b-1])) 
                - αw * intg.quad(lambda t : Uw(t) * Fm.pdf(t), x[Bm+b], x[Bm+b-1])[0])
        
    # PMF unity sum conditions
    E[Bm+Bw] = sum(Pbm)-1
    E[Bm+Bw+1] = sum(Pbw)-1
    
    # PMF non-negativity conditions
    for b in range(Bm):
        E[Bm+Bw+2+b] = Pbm[b]-abs(Pbm[b])
        
    for b in range(Bw): 
        E[Bm+Bw+2+Bm+b] = Pbw[b]-abs(Pbw[b])    
        
    return E 

## Solving For Steady State Equilibria 

In [7]:
reset_exog_params()
m_test = np.random.rand(Bm)#*0.5
w_test = np.random.rand(Bw)#*0.5  

print('μ0: ', m_test)
print('ω0: ', w_test)
print('')

x_start = np.concatenate((m_test, w_test), axis=None)
solution = least_squares(SSE, x_start, bounds = (0,1), verbose=1) 

μ_star = solution.x[:Bm]
ω_star = solution.x[Bm:]
print('')
print('μ*', μ_star) 
print('ω*', ω_star) 
print('Loss:', round(solution.cost,6))

μ0:  [0.92540542 0.81430504 0.67822919 0.64138401 0.93604564 0.35960761
 0.77870547 0.79789954 0.9258518  0.50416931]
ω0:  [0.99475944 0.83559118 0.39852463 0.02581193 0.42632547 0.23282404
 0.24543032 0.68953345 0.94650784 0.98334361]

`gtol` termination condition is satisfied.
Function evaluations 9, initial cost 2.0156e-01, final cost 6.4999e-15, first-order optimality 7.31e-09.

μ* [0.66366993 0.59515954 0.54890115 0.51301637 0.48332374 0.45781327
 0.43534896 0.41522015 0.3969499  0.38020117]
ω* [0.66367011 0.59515969 0.54890127 0.51301646 0.48332381 0.45781332
 0.435349   0.41522019 0.39694994 0.38020121]
Loss: 0.0


In [8]:
# Computing steady state
Nm, Nw, Pbm, Pbw, τm, τw, αm, αw = steady_state(μ_star, ω_star, True)
print('Masses: ', round(Nm), round(Nw))
print('PMF check:', round(sum(Pbm),3), round(sum(Pbw),3))
print('Tightness: ', round(τm,2), round(τw,2))
print('Alphas: ', round(αm,2), round(αw,2)) 

ρm = sum([(1 - Fm.cdf(ω_star[b]))*Pbw[b] for b in range(Bw)])
ρw = sum([(1 - Fw.cdf(μ_star[b]))*Pbm[b] for b in range(Bm)])
print('Average Pr(right-swipe): ', round(ρm, 2), round(ρw, 2))

Masses:  798 798
PMF check: 1.0 1.0
Tightness:  1 1.0
Alphas:  0.97 0.97
Average Pr(right-swipe):  0.49 0.49


## Single Batch Agent-Based Simulation

## Gale-Shapley Matches 

## Directed Search Random Walking Ranks

## Single Batch Agent-Based Simulation